In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import missingno as msno
import matplotlib.pyplot as plt

# Function to dynamically identify health-related columns
def identify_health_labels(df, exclude_cols=['cow', 'date', 'hour', 'IN_ALLEYS', 'REST', 'EAT', 'ACTIVITY_LEVEL']):
    potential_health_cols = [col for col in df.columns if col not in exclude_cols]
    health_labels = [col for col in potential_health_cols 
                     if df[col].dropna().isin([0, 1]).all()]
    return health_labels

c:\Users\lamia\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
def load_dataset(file_path):
    df = pd.read_csv(file_path)
    df['date'] = pd.to_datetime(df['date'])
    return df

### Check Completeness per Cow

In [3]:
def check_completeness_per_cow(df):
    df['date'] = pd.to_datetime(df['date'])
    all_dates = pd.date_range(start=df['date'].min(), end=df['date'].max(), freq='D')
    cow_date_counts = df.groupby('cow')['date'].nunique()
    expected_days = len(all_dates)
    missing_days_per_cow = expected_days - cow_date_counts
    missing_days_dict = missing_days_per_cow[missing_days_per_cow > 0].to_dict()
    print("\nMissing days per cow:")
    print(missing_days_per_cow[missing_days_per_cow > 0])
    return {'missing_days_per_cow': missing_days_dict, 'total_missing_days': sum(missing_days_dict.values())}

### The number of hourly entries per cow per day

In [4]:
def count_hourly_entries(df):
    hourly_counts = df.groupby(['cow', 'date']).size().reset_index(name='hourly_records')
    print(hourly_counts)
    return hourly_counts

### Filter for days with exactly 24 hours records

In [5]:
def filter_full_days(df):
    hourly_counts = df.groupby(['cow', 'date']).size().reset_index(name='hourly_records')
    full_days = hourly_counts[hourly_counts['hourly_records'] == 24]
    print(f"\nDays with full days of records: {len(full_days)}")
    return full_days

### The number of full 24-hour days per cow

In [6]:
def count_full_days_per_cow(df):
    hourly_counts = df.groupby(['cow', 'date']).size().reset_index(name='hourly_records')
    full_days = hourly_counts[hourly_counts['hourly_records'] == 24]
    full_days_per_cow = full_days.groupby('cow').size().reset_index(name='full_24h_days')
    print("\nFull 24h days per cow:")
    print(full_days_per_cow)
    return full_days_per_cow

In [7]:
def calculate_percentage_complete_days(df):
    hourly_counts = df.groupby(['cow', 'date']).size().reset_index(name='hourly_records')
    full_days = hourly_counts[hourly_counts['hourly_records'] == 24]
    percentage = len(full_days) / len(hourly_counts) * 100
    print(f"Percentage of complete days: {percentage:.2f}%")
    return percentage

### Count how many 24h samples have less than 12 observations

In [8]:
def count_less_than_12_obs(df):
    hourly_counts = df.groupby(['cow', 'date']).size().reset_index(name='hourly_records')
    less_than_12_obs = hourly_counts[hourly_counts['hourly_records'] < 12]
    print(f"Number of cow-day combinations with less than 12 hourly records: {len(less_than_12_obs)}")
    if len(less_than_12_obs) > 0:
        print(less_than_12_obs)
    return less_than_12_obs

### Filter dataset to exclude cow-day combinations with less than 12 observations

In [9]:
def filter_less_than_12_obs(df):
    hourly_counts = df.groupby(['cow', 'date']).size().reset_index(name='hourly_records')
    less_than_12_obs = hourly_counts[hourly_counts['hourly_records'] < 12]
    df_filtered = df.copy()
    if len(less_than_12_obs) > 0:
        valid_days = hourly_counts[hourly_counts['hourly_records'] >= 12]
        df_filtered = pd.merge(df, valid_days[['cow', 'date']], on=['cow', 'date'], how='inner')
        print(df_filtered)
    return df_filtered

### Perform cleaning by keeping only records with more than 18 samples

In [10]:
def clean_by_observation_count(file_path, min_obs=18, output_path='filtered_dataset_more_than_18_obs.csv'):
    df_cleaned = pd.read_csv(file_path)
    df_cleaned['date'] = pd.to_datetime(df_cleaned['date'])

    hourly_counts = df_cleaned.groupby(['cow', 'date']).size().reset_index(name='hour_count')
    records_more_than_18_obs = hourly_counts[hourly_counts['hour_count'] > min_obs]
    records_12_to_18_obs = hourly_counts[(hourly_counts['hour_count'] >= 12) & (hourly_counts['hour_count'] <= min_obs)]
    records_less_than_12_obs = hourly_counts[hourly_counts['hour_count'] < 12]

    df_more_than_18_obs = df_cleaned.merge(records_more_than_18_obs[['cow', 'date']], on=['cow', 'date'])
    df_12_to_18_obs = df_cleaned.merge(records_12_to_18_obs[['cow', 'date']], on=['cow', 'date'])
    df_less_than_12_obs = df_cleaned.merge(records_less_than_12_obs[['cow', 'date']], on=['cow', 'date'])

    df_filtered = df_cleaned[~df_cleaned.set_index(['cow', 'date']).index.isin(df_less_than_12_obs.set_index(['cow', 'date']).index)]

    print("Original dataset shape:", df_cleaned.shape)
    print(f"After filtering (<12 obs removed):", df_filtered.shape)
    print(f"Deleted rows (<12 obs):", df_less_than_12_obs.shape)
    print(f"Filtered dataset (>{min_obs} obs):", df_more_than_18_obs.shape)
    print(f"Filtered dataset (12–{min_obs} obs):", df_12_to_18_obs.shape)

    df_more_than_18_obs.to_csv(output_path, index=False)
    return df_more_than_18_obs, df_filtered, df_12_to_18_obs, df_less_than_12_obs, {
        'original_shape': df_cleaned.shape,
        'filtered_shape': df_filtered.shape,
        'deleted_rows': df_less_than_12_obs.shape,
        'more_than_18_shape': df_more_than_18_obs.shape,
        '12_to_18_shape': df_12_to_18_obs.shape
    }

### Perform another cleaning by keeping only physiological classes

Retain: mastitis, lameness, oestrus, calving, other_disease, OK.

Remove: management_changes, mixing, disturbance, accidents, LPS, acidosis.

In [11]:
# def clean_by_physiological_classes(file_path, output_path='dataset3_aligned_cleaned_keep_physiological.csv', unwanted_classes=['management_changes', 'mixing', 'disturbance', 'accidents', 'lps', 'acidosis']):
#     df = pd.read_csv(file_path)
#     df.columns = df.columns.str.strip().str.lower()

#     health_labels = identify_health_labels(df)
#     useful_classes = [col for col in health_labels if col not in unwanted_classes]

#     keep_cols = ['cow', 'date', 'hour', 'IN_ALLEYS', 'REST', 'EAT', 'ACTIVITY_LEVEL'] + useful_classes
#     keep_cols = [col for col in keep_cols if col in df.columns]
#     df_cleaned = df[keep_cols]

#     print(df_cleaned.head())
#     df_cleaned.to_csv(output_path, index=False)
#     return df_cleaned, {'kept_columns': keep_cols, 'removed_columns': [col for col in df.columns if col not in keep_cols]}
def clean_by_physiological_classes(
    file_path,
    output_path='dataset3_aligned_cleaned_keep_physiological.csv',
    unwanted_classes=[ 'lps', 'acidosis']
):
    df = pd.read_csv(file_path)
    df.columns = df.columns.str.strip().str.lower()

    # Identify all physiological class columns
    health_labels = identify_health_labels(df)

    # Keep only useful (wanted) physiological classes
    useful_classes = [col for col in health_labels if col not in unwanted_classes]

    # Define behavioral columns to retain
    behavior_cols = ['in_alleys', 'rest', 'eat', 'activity_level']

    # Define base columns
    base_cols = ['cow', 'date', 'hour']

    # Final list of columns to keep
    keep_cols = base_cols + behavior_cols + useful_classes
    keep_cols = [col for col in keep_cols if col in df.columns]  # filter out missing ones

    # Filter the DataFrame
    df_cleaned = df[keep_cols]

    print(df_cleaned.head())

    # Save cleaned dataset
    df_cleaned.to_csv(output_path, index=False)

    return df_cleaned, {
        'kept_columns': keep_cols,
        'removed_columns': [col for col in df.columns if col not in keep_cols]
    }


In [12]:
# # Function to create summary tables
# def create_summary_tables(summary_data):
#     # Dataset Shapes Table
#     shapes_df = pd.DataFrame({
#         'Dataset': [d['name'] for d in summary_data],
#         'Original Shape (Rows, Cols)': [f"{d['shapes']['original_shape'][0]}, {d['shapes']['original_shape'][1]}" for d in summary_data],
#         'Filtered Shape (<12 obs removed)': [f"{d['shapes']['filtered_shape'][0]}, {d['shapes']['filtered_shape'][1]}" for d in summary_data],
#         'Deleted Rows (<12 obs)': [f"{d['shapes']['deleted_rows'][0]}, {d['shapes']['deleted_rows'][1]}" for d in summary_data],
#         '>18 Obs Shape': [f"{d['shapes']['more_than_18_shape'][0]}, {d['shapes']['more_than_18_shape'][1]}" for d in summary_data],
#         '12–18 Obs Shape': [f"{d['shapes']['12_to_18_shape'][0]}, {d['shapes']['12_to_18_shape'][1]}" for d in summary_data]
#     })

#     # Completeness Metrics Table
#     completeness_df = pd.DataFrame({
#         'Dataset': [d['name'] for d in summary_data],
#         'Total Missing Days': [d['completeness']['total_missing_days'] for d in summary_data],
#         'Cows with Missing Days': [len(d['completeness']['missing_days_per_cow']) for d in summary_data],
#         'Full 24h Days': [d['full_days_count'] for d in summary_data],
#         'Percentage Complete Days': [f"{d['percentage_complete']:.2f}%" for d in summary_data],
#         'Cow-Days with <12 Obs': [len(d['less_than_12_obs']) for d in summary_data]
#     })

#     # Physiological Classes Table
#     phys_classes_df = pd.DataFrame({
#         'Dataset': [d['name'] for d in summary_data],
#         'Kept Columns': [', '.join(d['phys_classes']['kept_columns']) for d in summary_data],
#         'Removed Columns':[ ['removed_columns'] for d in summary_data]
#     })

#     return shapes_df, completeness_df, phys_classes_df
def create_summary_tables(summary_data):
    # Dataset Shapes Table
    shapes_df = pd.DataFrame({
        'Dataset': [d['name'] for d in summary_data],
        'Original Shape (Rows, Cols)': [f"{d['shapes']['original_shape'][0]}, {d['shapes']['original_shape'][1]}" for d in summary_data],
        'Filtered Shape (<12 obs removed)': [f"{d['shapes']['filtered_shape'][0]}, {d['shapes']['filtered_shape'][1]}" for d in summary_data],
        'Deleted Rows (<12 obs)': [f"{d['shapes']['deleted_rows'][0]}, {d['shapes']['deleted_rows'][1]}" for d in summary_data],
        '>18 Obs Shape': [f"{d['shapes']['more_than_18_shape'][0]}, {d['shapes']['more_than_18_shape'][1]}" for d in summary_data],
        '12–18 Obs Shape': [f"{d['shapes']['12_to_18_shape'][0]}, {d['shapes']['12_to_18_shape'][1]}" for d in summary_data]
    })

    # Completeness Metrics Table
    completeness_df = pd.DataFrame({
        'Dataset': [d['name'] for d in summary_data],
        'Total Missing Days': [d['completeness']['total_missing_days'] for d in summary_data],
        'Cows with Missing Days': [len(d['completeness']['missing_days_per_cow']) for d in summary_data],
        'Full 24h Days': [d['full_days_count'] for d in summary_data],
        'Percentage Complete Days': [f"{d['percentage_complete']:.2f}%" for d in summary_data],
        'Cow-Days with <12 Obs': [len(d['less_than_12_obs']) for d in summary_data]
    })

    # Physiological Classes Table
    phys_classes_df = pd.DataFrame({
        'Dataset': [d['name'] for d in summary_data],
        'Kept Columns': [', '.join(d['phys_classes']['kept_columns']) for d in summary_data],
        'Removed Columns': [', '.join(d['phys_classes']['removed_columns']) for d in summary_data]
    })

    return shapes_df, completeness_df, phys_classes_df


In [13]:
datasets = [
    r"C:/Users/lamia/Desktop/datasets/dataset1.csv",
    r"C:/Users/lamia/Desktop/datasets/dataset2.csv",
    r"C:/Users/lamia/Desktop/datasets/dataset3.csv",
    r"C:/Users/lamia/Desktop/datasets/dataset4_truncated.csv"
]

summary_data = []

for file_path in datasets:
    print(f"\nAnalyzing {file_path}")
    df = load_dataset(file_path)

    dataset_summary = {'name': file_path.split('/')[-1]}

    print("\nCheck Completeness per Cow:")
    dataset_summary['completeness'] = check_completeness_per_cow(df)

    print("\nHourly Entries per Cow per Day:")
    dataset_summary['hourly_counts'] = count_hourly_entries(df)

    print("\nDays with Exactly 24 Hours Records:")
    dataset_summary['full_days'] = filter_full_days(df)
    dataset_summary['full_days_count'] = len(dataset_summary['full_days'])

    print("\nFull 24-Hour Days per Cow:")
    dataset_summary['full_days_per_cow'] = count_full_days_per_cow(df)

    print("\nPercentage of Complete Days:")
    dataset_summary['percentage_complete'] = calculate_percentage_complete_days(df)

    print("\nCow-Day Combinations with Less than 12 Hourly Records:")
    dataset_summary['less_than_12_obs'] = count_less_than_12_obs(df)

    print("\nFiltered Dataset (Excluding <12 obs):")
    dataset_summary['filtered_less_than_12'] = filter_less_than_12_obs(df)

    print("\nCleaning by Observation Count (>18 obs):")
    output_path_obs = f"{file_path.split('.')[0]}_more_than_18_obs.csv"
    df_more_than_18, df_filtered_obs, df_12_to_18, df_less_than_12, shapes_info = clean_by_observation_count(file_path, output_path=output_path_obs)
    dataset_summary['shapes'] = shapes_info

    print("\nCleaning by Physiological Classes:")
    output_path_phys = f"{file_path.split('.')[0]}_physiological.csv"
    df_phys_cleaned, phys_info = clean_by_physiological_classes(file_path, output_path=output_path_phys)
    dataset_summary['phys_classes'] = phys_info

    summary_data.append(dataset_summary)

# Display summary tables
print("\n=== Summary Tables for All Datasets ===\n")
shapes_df, completeness_df, phys_classes_df = create_summary_tables(summary_data)

print("Dataset Shapes Summary:")
display(shapes_df)

print("\nCompleteness Metrics Summary:")
display(completeness_df)

print("\nPhysiological Classes Summary:")
display(phys_classes_df)


Analyzing C:/Users/lamia/Desktop/datasets/dataset1.csv

Check Completeness per Cow:

Missing days per cow:
cow
6601     3
6610     3
6612     3
6613    30
6621     3
6629     3
6633     3
6634     3
6637     3
6638    18
6643    18
6646     3
6656     3
6664    19
6674    19
6675     3
6683    19
6686    18
6689     3
6690     3
6693    19
6695    38
6699    29
6701    18
6714    18
6721    30
6750    18
7600    19
Name: date, dtype: int64

Hourly Entries per Cow per Day:
       cow       date  hourly_records
0     6601 2018-10-25              13
1     6601 2018-10-26              24
2     6601 2018-10-27              24
3     6601 2018-10-28              24
4     6601 2018-10-29              24
...    ...        ...             ...
4526  7600 2019-04-13              24
4527  7600 2019-04-14              24
4528  7600 2019-04-15              24
4529  7600 2019-04-16              24
4530  7600 2019-04-17              24

[4531 rows x 3 columns]

Days with Exactly 24 Hours Records:

Day

,Dataset,"Original Shape (Rows, Cols)",Filtered Shape (<12 obs removed),Deleted Rows (<12 obs),>18 Obs Shape,12–18 Obs Shape
0,dataset1.csv,"107665, 19","107665, 19","0, 19","106269, 19","1396, 19"
1,dataset2.csv,"40247, 19","40247, 19","0, 19","40247, 19","0, 19"
2,dataset3.csv,"26225, 19","26223, 19","2, 19","26185, 19","38, 19"
3,dataset4_truncated.csv,"179759, 19","179759, 19","0, 19","179759, 19","0, 19"



Completeness Metrics Summary:


,Dataset,Total Missing Days,Cows with Missing Days,Full 24h Days,Percentage Complete Days,Cow-Days with <12 Obs
0,dataset1.csv,369,28,4315,95.23%,0
1,dataset2.csv,3,1,1676,99.94%,0
2,dataset3.csv,164,12,1078,98.36%,1
3,dataset4_truncated.csv,2410,25,7489,99.99%,0



Physiological Classes Summary:


,Dataset,Kept Columns,Removed Columns
0,dataset1.csv,"cow, date, hour, in_alleys, rest, eat, activit...","lps, acidosis"
1,dataset2.csv,"cow, date, hour, in_alleys, rest, eat, activit...","lps, acidosis"
2,dataset3.csv,"cow, date, hour, in_alleys, rest, eat, activit...","lps, acidosis"
3,dataset4_truncated.csv,"cow, date, hour, in_alleys, rest, eat, activit...","lps, acidosis"
